In [5]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


##Cibersort

In [29]:
PATH='/cellar/users/ramarty/Data/hla_ii/clinical/immune_infiltration/'
files = [PATH+x for x in os.listdir(PATH)]

In [30]:
all_dfs = []
for f in files:
    all_dfs.append(pd.read_csv(f, index_col=0))
df = pd.concat(all_dfs)

In [31]:
df.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/cibersort.csv')

Division into hot/cold based on CD4

In [32]:
df.columns

Index([u'B cells naive', u'B cells memory', u'Plasma cells', u'T cells CD8',
       u'T cells CD4 naive', u'T cells CD4 memory resting',
       u'T cells CD4 memory activated', u'T cells follicular helper',
       u'T cells regulatory (Tregs)', u'T cells gamma delta',
       u'NK cells resting', u'NK cells activated', u'Monocytes',
       u'Macrophages M0', u'Macrophages M1', u'Macrophages M2',
       u'Dendritic cells resting', u'Dendritic cells activated',
       u'Mast cells resting', u'Mast cells activated', u'Eosinophils',
       u'Neutrophils', u'P-value', u'Pearson Correlation', u'RMSE'],
      dtype='object')

In [33]:
df['CD4'] = df['T cells CD4 naive'] + df['T cells CD4 memory resting'] \
            + df['T cells CD4 memory activated']

In [34]:
conservative_patients = list(pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.TCGA.conservative.mut.ClassI.csv',
            index_col=0).index)

In [35]:
df = df.loc[conservative_patients, 'CD4'].dropna()

In [42]:
cold = list(df[df <= df.quantile(.25)].index)
cool = list(df[(df > df.quantile(.25))&(df <= df.quantile(.5))].index)
warm = list(df[(df > df.quantile(.5))&(df <= df.quantile(.75))].index)
hot = list(df[df > df.quantile(.75)].index)

In [44]:
for l, heat in zip([cold, cool, warm, hot], ['cold', 'cool', 'warm', 'hot']):
    with open('/cellar/users/ramarty/Data/hla_ii/generated_data/clinical/CD4_infiltration.{0}.txt'.format(heat), 'w') as f:
        for p in l:
            f.write('{0}\n'.format(p))

##Cytotoxicity

In [36]:
PATH='/cellar/users/ramarty/Data/hla_ii/clinical/expression/'
files = [PATH+x for x in os.listdir(PATH)]

In [37]:
all_dfs = []
for f in files:
    try:
        all_dfs.append(pd.read_csv(f, sep='\t', index_col=0).transpose()[['GZMA', 'PRF1']])
    except:
        print f
df = pd.concat(all_dfs)

/cellar/users/ramarty/Data/hla_ii/clinical/expression/expression.6000_7000.tsv


In [38]:
df['Cytolytic'] = sp.mstats.gmean(df[['GZMA', 'PRF1']], axis=1)

In [39]:
df.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/cytotoxicity.csv')

In [34]:
df.shape

(8658, 3)

In [24]:
# missing some genes
test = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/expression/expression.6000_7000.tsv', sep='\t',
                  index_col=0).transpose()

In [33]:
test[['GZMA']].head()

symbol,GZMA
TCGA-A7-A26H,359
TCGA-VD-A8KD,622
TCGA-24-1616,80
TCGA-56-8503,214
TCGA-CJ-4874,2338
